In [19]:
import BSgeneration
import GetUsers
import UserMobility
import PathLossCalc, CorrelatedShadowing, DirectivityGain
import ChannelGainCalculation, SINRCalculation

In [20]:
radius = 500

In [21]:
bsgen = BSgeneration.BSGenerate(radius)

In [22]:
X, Y = bsgen.tier1cellgen()

TypeError: 'NoneType' object is not iterable

In [23]:
X

NameError: name 'X' is not defined

In [14]:
BS_wrap_around_total3x, BS_wrap_around_total3y = bsgen.WrapAround()

TypeError: 'NoneType' object is not iterable

In [ ]:
def SINRsimulation(each_sec, Time_instants, v_min, v_max, std_dev):  #nums = total no.of simulations, each_sec = no. of users in each sector
    
    num_users, x, y = GetUsers.GetSecusers(each_sec)  #<----- each_sec amount of users in each sector for 1 iteration
    #PlotSecUsers(num_users, x, y)

    x_mobile, y_mobile, users_velocity = UserMobility.User_Mobility(num_users, x, y, v_min, v_max, Time_instants) #time_instants x total_users

    #doing calculations only for the centre cell
    num_users = each_sec*3
    x_mobile = x_mobile[:, :num_users]
    y_mobile = y_mobile[:, :num_users]

    dist2d, dist3d = PathLossCalc.getdist(num_users, x_mobile, y_mobile, Time_instants, BS_wrap_around_total3x, BS_wrap_around_total3y)

    path_loss_wrap = PathLossCalc.PathLoss(dist2d, dist3d, num_users) #this path loss considers 3 base station in each hex cell


    #shadowing = LogNormShadow(path_loss_wrap)
    #print(shadowing)

    #shadowing = getshadloss(x_mobile, y_mobile)

    shadowing = correlated_shadowing(x_mobile, y_mobile, std_dev)
    #shadowing = np.zeros(np.shape(path_loss_wrap))

    #print(shadowing)

    angles_users = UserAngle(x_mobile, y_mobile, BS_wrap_around_total3x, BS_wrap_around_total3y)

    #channel_gain_linear = ChannelGainCalculation(path_loss_wrap, shadowing, angles_users)
    channel_gain_linear = ChannelGainCalculation(path_loss_wrap, shadowing, angles_users)

    channel_gain_dB = 10*np.log10(channel_gain_linear)
    #print(channel_gain_dB)
    #print(np.shape(channel_gain_linear))

    Sinr_dB, sp, interf = SINR_b4(channel_gain_linear)      # time_steps x 21 x 210            


    plt.show()


    return Sinr_dB, x_mobile, y_mobile, shadowing, channel_gain_dB, sp, interf